# The Monte Carlo Module

In [1]:
# A code block with your classes.
import pandas as pd
import numpy as np


class Die:
    '''A die has N sides, or “faces”, and W weights, and can be rolled to select a face.

       W defaults to 1.0 for each face but can be changed after the object is created.
       Note that the weights are just numbers, not a normalized probability distribution.
       The die has one behavior, which is to be rolled one or more times.
       Note that what we are calling a “die” here can represent a variety of random variables associated with stochastic processes, such as using a deck of cards or flipping a coin or speaking a language. 
       We can create these models by increasing the number of sides and defining the values of their faces. 
       Our probability models for such variables are, however, very simple – since our weights apply to only to single events, we are assuming that the events are independent.'''
    def __init__(self,faces):
        '''Initializer takes an array of faces as an argument.
        The array's data type (dtype) may be strings or numbers.
        The faces must be unique; no duplicates.
        Internally iInitializes the weights to 1.0 for each face.
        Saves faces and weights in a private dataframe that is to be shared by the other methods.'''
    
        self.faces = list(set(faces)) # array of strings or numbers
        self.weights =  [1.0 for self.face in range(len(self.faces))] # array of integers
        self.df = pd.DataFrame({'faces':self.faces, 'weights':self.weights}) #dataframe with faces and weights
        
    def change_weight(self,face_value,new_weight):
        '''This method is to change the weight of a single side.
        Takes two arguments: the face value to be changed and the new weight.
        Checks to see if the face passed is valid; is it in the array of weights?
        Checks to see if the weight is valid; is it a float? Can it be converted to one?'''
        self.checks = []
        if (type(face_value)==str or type(face_value)==int) and face_value in self.faces:
            self.checks.append(True)
#             print("Checks passed,valid face value and weight")
        else:
            self.checks.append(False)
            print("Face value should be array of strings or integer.  ")
            
        try:
            self.weight=float(new_weight)
            self.checks.append(True)
        except:
            print("Weight cannot be converted to float")
            self.checks.append(False)
        
        if self.checks:
            self.i = self.faces.index(face_value)
            self.weights[self.i] = new_weight
        else:
            print("Checks failed. Pass valid face value and weight")
            
        self.df = pd.DataFrame({'faces':self.faces, 'weights':self.weights})            
        return self.df
        
    def roll(self,num_roll=1):
        '''This method is to roll the die one or more times.
        Takes a parameter of how many times the die is to be rolled; defaults to 1.
        This is essentially a random sample from the vector of faces according to the weights.
        Returns a list of outcomes.
        Does not store internally these results.'''    
        return list(self.df.sample(n=num_roll,weights='weights')['faces'])
         
    def show(self):
        '''This method is to show the user the die’s current set of faces and weights (since the latter can be changed).
        Returns the dataframe created in the initializer but possibly updated by the weight changing method.'''
        return self.df
    
    
    
class Game:
    '''Game class consists of rolling of one or more dice of the same kind one or more times.
       Each game is initialized with a list of one or more of similarly defined dice (Die objects).
       By “same kind” and “similarly defined” we mean that each die in a given game has the same number of sides and set of faces, but each die object may have its own weights.
       The class has a behavior to play a game, i.e. to roll all of the dice a given number of times.
       The class keeps the results of its most recent play.'''
    
    
    def __init__(self,dice_list):
        '''Takes a single parameter, a list of already instantiated similar Die objects.'''
        self.dice_list = dice_list
    
    def play(self,face_list,num_roll):
        '''Takes a parameter to specify how many times the dice should be rolled. Saves the result of the play to a private dataframe of shape N rolls by M dice.
        That is, each role is an observation and each column is a feature. Each cell should show the resulting face for the die on the roll. Note that this table is in wide form.
        The private dataframe should have the roll number is a named index.'''
        self.num_roll = num_roll
        self.face_list = face_list
        self.play_result = pd.DataFrame()
        n=1
        for die in self.dice_list:
            die_class = Die(face_list)
            self.result = die_class.roll(num_roll)
            self.play_result[die] = self.result
            n+= 1
        
        self.play_result['Game']=list(np.arange(1,num_roll+1))
        self.result_df = self.play_result.set_index('Game')
        return self.result_df
            
    def show(self,df_form='wide'):
        '''This method just passes the private dataframe to the user.
        Takes a parameter to return the dataframe in narrow or wide form.
        This parameter defaults to wide form, which is what the previously described method produces.
        This parameter should raise an exception if the user passes an invalid option.
        The narrow form of the dataframe will have a two-column index with the roll number and the die number, and a single column for the face rolled.
        As a reminder, the wide form of the dataframe will a have single column index with the roll number, and each die number as a column.'''
        
        self.df_form = df_form
        if self.df_form == 'wide':
#             print("wide")
            self.df_to_return = self.result_df
        elif df_form == 'narrow':   
            self.df_to_return = self.result_df.stack().to_frame('Face')
        else:
            raise Exception("Dataframe form should be wide or narrow")
        return self.df_to_return
    
    
class Analyzer:
    def __init__(self,game):
        '''Takes a game object as its input parameter.At initialization time, it also infers the data type of the die faces used.'''
        self.game = game
    
    def face_counts_per_roll(self,game):
        '''A face counts per roll method to compute how many times a given face is rolled in each event.
        Stores the results as a dataframe in a public attribute.
        The dataframe has an index of the roll number and face values as columns (i.e. it is in wide format).'''
        face_counts_per_roll = pd.DataFrame()
        for i in range(len(game)):
            each_roll = pd.DataFrame(game.iloc[i].value_counts()).transpose()
            face_counts_per_roll = face_counts_per_roll.append(each_roll)
        face_counts_per_roll.index.name = 'Game'
        return face_counts_per_roll
    
    def combo(self):
        '''A combo method to compute the distinct combinations of faces rolled, along with their counts.
        Combinations should be sorted and saved as a multi-columned index.
        Stores the results as a dataframe in a public attribute.
        Note that this class helps compute the jackpot, since a jackpot is a combination in which only one face appears.'''
        self.combo_results = pd.DataFrame()
        for i in range(len(self.game)):
            self.combo_results = self.combo_results.append(pd.DataFrame(self.game.iloc[i].value_counts()).transpose())
            self.combo_results.index.name = "Game"
        return self.combo_results
                                     
    
    def jackpot(self):
        '''A jackpot method to compute how many times the game resulted in all faces being identical.
        Returns an integer for the number times to the user.
        Stores the results as a dataframe of jackpot results in a public attribute.
        The dataframe should have the roll number as a named index.'''
        jackpot_results = pd.DataFrame()
        analyzer = Analyzer(self.game)
        self.combo_results = analyzer.combo()
        for i in range(len(self.combo_results)):
            if self.combo_results.iloc[i].count()==1:
                jackpot_results = jackpot_results.append(self.combo_results.iloc[i])
        all_faces_identical = len(jackpot_results)
        return all_faces_identical
          

# Test Module

In [3]:
# A code block with your test code.
import unittest
import numpy as np

class MonteCarloTestSuite(unittest.TestCase):
    def test_test(self):
        self.assertTrue(True)
        
    # 'test_1_change_weight()': Change weight of a face in list of faces.
    # 'test_2_roll_die()': Roll die 2 times and store results in dataframe
    # 'test_3_die_show()': Show faces of die
    # 'test_4_game_play()': Play game twice and see results
    # 'test_5_face_counts()': Count faces of game
    # 'test_6_combo()': Verify if the combos are summed
    # 'test_7_jackpot()': Verify jackpot results are correct

    
    def test_1_die_change_weight(self):
        # Change weight of a face
        faces_list = [2,3,4,5]
        mc_change_weight = Die(faces_list)
        df = mc_change_weight.change_weight(2,6)
        assert (df['weights'][0] == 6)
        
    def test_2_die_roll(self):
        # Roll die 2 times and store results in dataframe
        faces_list = [2,3,4,5]
        die = Die(faces_list)
        roll_list = die.roll(2)
        assert (len(roll_list) == 2)
        
    def test_3_die_show(self):
        # Roll die 2 times and store results in dataframe
        faces_list = [2,3,4,5]
        die = Die(faces_list)
        df = die.show()
        assert (len(df) == 4)
                  
    def test_4_game_play(self):
        # Play game and store the results in a wide dataframe
        dice_list = ['die1','die2','die3','die4']
        faces_list = [2,3,4,5]
        game = Game(dice_list)
        game_play = game.play(faces_list,4)
        game.show("wide")
        assert len(game_play)==4
               
    def test_5_face_counts(self):
        # Count faces of game
        df = pd.DataFrame({'Game':[1,2,3,4], 'die1':[2,6,4,7], 'die2':[4,8,4,2], 'die3':[1,5,4,6]})
        df = df.set_index('Game')
        analyze = Analyzer(df)
        face_counts = analyze.face_counts_per_roll(df)
        assert (face_counts.iloc[3][6] ==1)
      
    def test_6_combo(self):
        # Verify if the combos are summed
        df = pd.DataFrame({'Game':[1,2,3,4], 'die1':[2,6,4,7], 'die2':[4,8,4,2], 'die3':[1,5,4,6]})
        df = df.set_index('Game')
        analyze = Analyzer(df)
        game_combo = analyze.combo()
        assert (game_combo.iloc[2][4] == 3)
        
    def test_7_jackpot(self):
        #Verify jackpot results are correct
        df = pd.DataFrame({'Game':[1,2,3,4], 'die1':[2,6,4,7], 'die2':[4,8,4,2], 'die3':[1,5,4,6]})
        df = df.set_index('Game')
        analyze = Analyzer(df)
        game_combo = analyze.jackpot()
        assert (game_combo == 1)
        
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

........
----------------------------------------------------------------------
Ran 8 tests in 0.058s

OK


# Test Results

# A text block with the output of a successful test.
........
----------------------------------------------------------------------
Ran 8 tests in 0.053s

OK

# Scenarios

Code blocks with your scenarios and their outputs. 

These should have appropriate import statements even though the code is now in the same notebook as the classes it calls. 

## Scenario 1

In [169]:
# Code blocks with output

def scenario_1(dice_list,faces, num_roll):
    '''Roll D dice with N faces X times and check if any game resulted in Jackpot'''
    game = Game(dice_list)
    game_play = game.play(faces,num_roll)
    game_play_narrow = game.show('narrow')
    print(game_play_narrow)
    
    analyze = Analyzer(game_play)
    jackpot_analyze =  analyze.jackpot()
    print("Number of Jackpots : " + str(jackpot_analyze))

    
# #Test: Roll 4 dice with 6 faces 3 times and check if any game resulted in Jackpot'''
# faces = [1,2,3,4,5,6]
# dice_list = ['Die1','Die2','Die3','Die4']
# num_roll = 3

# scenario_1(dice_list,faces,num_roll)

#Test: Roll 2 dice with 6 faces 5 times and check if any game resulted in Jackpot'''
faces = [1,2,3,4,5,6]
dice_list = ['Die1','Die2']
num_roll = 4

scenario_1(dice_list,faces,num_roll)

           Face
Game           
1    Die1     4
     Die2     4
2    Die1     1
     Die2     1
3    Die1     6
     Die2     5
4    Die1     2
     Die2     6
Number of Jackpots : 2


## Scenario 2

In [170]:
# Code blocks with output
def scenario_2(dice_list,faces, num_roll):
    '''Roll D dice with N faces X times and analyze the combination'''
    game = Game(dice_list)
    game_play = game.play(faces,num_roll)
    print(game_play)
    
    analyze = Analyzer(game_play)
    combo_analyze =  analyze.combo().fillna(0)
    print(combo_analyze)

    
#Test: Roll 12 dice with 12 faces 3 times and analyze the combination'''
faces = [1,2,3,4,5,6,7,8,9,10,11,12]
dice_list = ['Die1','Die2','Die3','Die4','Die5','Die6','Die7','Die8','Die9','Die10','Die11','Die12']
num_roll = 12

scenario_2(dice_list,faces,num_roll)

      Die1  Die2  Die3  Die4  Die5  Die6  Die7  Die8  Die9  Die10  Die11  \
Game                                                                       
1       10     4     7     6     6    10     6    10     8      4      6   
2       12     3     4    11    11     5     8     3     7      2     11   
3        6    10     5    10     4     7     1    12     9      1      1   
4        7     1     1     2     5    12     5     2     4     10      4   
5        1     9     3     7     2     4    10    11    12      5      9   
6        4     6    10     5     1     9     3     9     6      7      2   
7        2     5     9    12     3     3    11     7     2      6      5   
8        9    11    11     3     9     6     7     6     1      3     12   
9        8     2     6     9    12     1     2     4     5      8     10   
10       3    12     8     1     7     8    12     5    10      9      3   
11      11     8    12     8    10    11     4     1     3     12      7   
12       5  

## Scenario 3

In [173]:
# Code blocks with output
def scenario_3(dice_list,faces, num_roll):
    '''Roll D dice with N faces X times and print results in narrow dataframe. Check for jackpot'''
    game = Game(dice_list)
    game_play = game.play(faces,num_roll)
    print(game.show('narrow'))
    
   
    analyze = Analyzer(game_play)
    jackpot_analyze =  analyze.jackpot()
    print("Number of Jackpots : " + str(jackpot_analyze))

    
#Test: Roll 12 dice with 12 faces 3 times and print results in narrow dataframe. Check for jackpot'''
faces = [1,2,3,4,5,6,7,8,9,10]
dice_list = ['Die1','Die2','Die3','Die4','Die5','Die6']
num_roll = 3

scenario_3(dice_list,faces,num_roll)

           Face
Game           
1    Die1     8
     Die2     7
     Die3    10
     Die4     2
     Die5     9
     Die6     7
2    Die1     7
     Die2     1
     Die3     9
     Die4     9
     Die5     8
     Die6     2
3    Die1     5
     Die2     3
     Die3     5
     Die4     7
     Die5    10
     Die6    10
Number of Jackpots : 0


# Directory Listing

A code block that executes the following bash command: 

```bash
!ls -lRF -o
```

In [8]:
!ls -lRF -o

.:
total 64
-rw-r--r-- 1 nyc2xu 30442 Nov 27 12:58 montecarlo_demo.ipynb
-rw-r--r-- 1 nyc2xu   957 Nov 27 13:12 montecarlo_test_results.txt
-rw-r--r-- 1 nyc2xu  2726 Nov 27 13:11 montecarlo_tests.py
drwxr-sr-x 3 nyc2xu  2560 Nov 27 13:08 pkg_mc/
drwxr-sr-x 2 nyc2xu  2560 Nov 27 12:57 pkg_mc.egg-info/
-rw-r--r-- 1 nyc2xu   772 Nov 27 13:15 README.md
-rw-r--r-- 1 nyc2xu   318 Nov 27 12:57 setup.py

./pkg_mc:
total 40
-rw-r--r-- 1 nyc2xu     0 Nov 27 12:51 __init__.py
-rw-r--r-- 1 nyc2xu   190 Nov 27 13:02 __init__.pyc
-rw-r--r-- 1 nyc2xu  8637 Nov 27 13:07 montecarlo.py
-rw-r--r-- 1 nyc2xu 10310 Nov 27 13:07 montecarlo.pyc

./pkg_mc.egg-info:
total 20
-rw-r--r-- 1 nyc2xu   1 Nov 27 12:57 dependency_links.txt
-rw-r--r-- 1 nyc2xu 238 Nov 27 12:57 PKG-INFO
-rw-r--r-- 1 nyc2xu  15 Nov 27 12:57 requires.txt
-rw-r--r-- 1 nyc2xu 207 Nov 27 12:57 SOURCES.txt
-rw-r--r-- 1 nyc2xu   7 Nov 27 12:57 top_level.txt


# Installation Output Listing
    
A code block that executes the code to install your your package and outputs a successful installation.

In [7]:
# Installation commands
!pip install -e .

Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///sfs/qumulo/qhome/nyc2xu/Documents/MSDS/DS5100/DS5100-2022-08-nyc2xu/Project/project
  Running setup.py develop for pkg-mc
